In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb  8 13:52:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    31W /  70W |    482MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

### Setup and Load dataset

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1gfGS5iL8-8HxdWVtTHXLGWGfPZRvMj6x


Downloading...
From: https://drive.google.com/uc?id=1gfGS5iL8-8HxdWVtTHXLGWGfPZRvMj6x
To: /content/Music_InCarMusic.zip
100% 152k/152k [00:00<00:00, 80.4MB/s]


In [ ]:
!unzip "Music_InCarMusic.zip"  -d  "/content"

Archive:  Music_InCarMusic.zip
replace /content/Music_InCarMusic/Data_InCarMusic.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/Music_InCarMusic/Data_InCarMusic.xlsx  
  inflating: /content/Music_InCarMusic/ReadMe.txt  


In [ ]:
!pip install pandas openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import pandas as pd

xls = pd.ExcelFile(r"Music_InCarMusic/Data_InCarMusic.xlsx") 

rating_df = xls.parse(0) #2 is the sheet number+1 thus if the file has only 1 sheet write 0 in paranthesis
music_df = xls.parse(2)
context_df = xls.parse(1)
cat_df = xls.parse(3)

In [ ]:
rating_df['mood'] = rating_df['mood'].fillna('NA')
rating_df['RoadType'] = rating_df['RoadType'].fillna('NA')
rating_df['sleepiness'] = rating_df['sleepiness'].fillna('NA')
rating_df['trafficConditions'] = rating_df['trafficConditions'].fillna('NA')
rating_df['weather'] = rating_df['weather'].fillna('NA')
rating_df['landscape'] = rating_df['landscape'].fillna('NA')
rating_df['DrivingStyle'] = rating_df['DrivingStyle'].fillna('NA')

rating_df['naturalphenomena '] = rating_df['naturalphenomena '].fillna('NA')
rating_df.head(1)

,UserID,ItemID,Rating,DrivingStyle,landscape,mood,naturalphenomena,RoadType,sleepiness,trafficConditions,weather
0,1001,715,2,NA,NA,NA,NA,NA,NA,NA,sunny


In [ ]:
rating_df=rating_df.reset_index(drop=True)

In [ ]:

rating_df.rename(columns = {'UserID':'userID'}, inplace = True)
rating_df.rename(columns = {'ItemID':'itemID'}, inplace = True)
rating_df.rename(columns = {'naturalphenomena ':'naturalphenomena'}, inplace = True)
rating_df.rename(columns = {' Rating':'rating'}, inplace = True)

In [ ]:
rating_df["itemID"] = rating_df["itemID"].astype(str)
rating_df["userID"] = rating_df["userID"].astype(str)
rating_df["mood"] = rating_df["mood"].astype(str)
rating_df["RoadType"] = rating_df["RoadType"].astype(str)
rating_df["sleepiness"] = rating_df["sleepiness"].astype(str)
rating_df["trafficConditions"] = rating_df["trafficConditions"].astype(str)
rating_df["weather"] = rating_df["weather"].astype(str)
rating_df["landscape"] = rating_df["landscape"].astype(str)
rating_df["DrivingStyle"] = rating_df["DrivingStyle"].astype(str)
rating_df["naturalphenomena"] = rating_df["naturalphenomena"].astype(str)

# Tensorflow recommenders

In [ ]:
!pip install tensorflow-recommenders==0.6.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs
from typing import Dict, Text

In [ ]:
interactions_dict = rating_df[['userID', 
                                'itemID',
                                 'DrivingStyle',
                                'landscape',
                                'naturalphenomena',
                                'RoadType',
                                'sleepiness',
                                'trafficConditions',
                                'weather',
                                'mood',
                                'rating'
                                      ]]


In [ ]:
ratings = tf.data.Dataset.from_tensor_slices(dict(interactions_dict))

In [ ]:
# #interactions_dict = {name: np.array(value) for name, value in interactions_dict.items()}
# interactions = tf.data.Dataset.from_tensor_slices(dict(interactions_dict))

# ## item features
# items_dict = rating_df[['itemID']].drop_duplicates()
# #items_dict = {name: np.array(value) for name, value in items_dict.items()}
# items = tf.data.Dataset.from_tensor_slices(dict(items_dict))

# ## map the features in interactions and items


In [ ]:

# # Select the basic features.
# interactions = interactions.map(lambda x: {
#     'userID' : str(x['userID']), 
#     'itemID' : str(x['itemID']), 
#     'DrivingStyle' : str(x['DrivingStyle']), 
#     'landscape' : str(x['landscape']), 
#     'naturalphenomena' : str(x['naturalphenomena']), 
#      'sleepiness' : str(x['sleepiness']), 
#     'RoadType' : str(x['RoadType']), 
#     'trafficConditions' : str(x['trafficConditions']), 
#      'weather' : str(x['weather']), 
#     'mood' : str(x['mood']), 
#     'rating' : int(x['rating']),
    
# })

# itemlists = items.map(lambda x: str(x['itemID']))

In [ ]:


## item features
items_dict = rating_df[['itemID']].drop_duplicates()
#items_dict = {name: np.array(value) for name, value in items_dict.items()}
items = tf.data.Dataset.from_tensor_slices(dict(items_dict))

## map the features in interactions and items

In [ ]:
itemlists = items.map(lambda x: x['itemID'])

In [ ]:
# itemlists = items.map(lambda x: x['itemID'])

In [ ]:
userIds    = rating_df.userID.unique()
productIds = rating_df.itemID.unique()
total_ratings= len(rating_df.index)

In [ ]:
unique_mood = np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["mood"]))))

In [ ]:

unique_landscape = np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["landscape"]))))
unique_DrivingStyle = np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x:x["DrivingStyle"]))))

unique_naturalphenomena = np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["naturalphenomena"]))))
unique_RoadType =np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["RoadType"]))))
unique_sleepiness =np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["sleepiness"]))))

unique_trafficConditions =np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["trafficConditions"]))))
unique_weather =np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["weather"]))))

unique_user_ids =np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["userID"]))))

In [ ]:
unique_items = np.unique(np.concatenate(list(items.batch(1000).map(lambda x: x["itemID"]))))

In [ ]:
total=ratings.__len__().numpy()
train_size=(int) (total * .8 )
test_size=(int) (total * .2 )

In [ ]:
total_ratings= len(rating_df.index)
tf.random.set_seed(42)
shuffled = ratings.shuffle(int(total_ratings), seed=42, reshuffle_each_iteration=False)

train = shuffled.take( int(total_ratings*0.8) )
test = shuffled.skip(int(total_ratings*0.8)).take(int(total_ratings*0.2))
# unique_productIds = unique_items
# unique_userIds    = unique_user_ids
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()


In [ ]:
# mood = np.concatenate(list(interactions.map(lambda x: x["mood"]).batch(100)))
# landscape = np.concatenate(list(interactions.map(lambda x: x["landscape"]).batch(100)))
# DrivingStyle = np.concatenate(list(interactions.map(lambda x: x["DrivingStyle"]).batch(100)))

# naturalphenomena = np.concatenate(list(interactions.map(lambda x: x["naturalphenomena"]).batch(100)))
# RoadType = np.concatenate(list(interactions.map(lambda x: x["RoadType"]).batch(100)))
# sleepiness = np.concatenate(list(interactions.map(lambda x: x["sleepiness"]).batch(100)))

# trafficConditions = np.concatenate(list(interactions.map(lambda x: x["trafficConditions"]).batch(100)))
# weather = np.concatenate(list(interactions.map(lambda x: x["weather"]).batch(100)))

In [ ]:
# itemlists=np.concatenate(list(items.map(lambda x: x["itemID"]).batch(100)))

#  Retrival

In [ ]:
class UserModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        self.embedding_dimension = 32
        max_tokens = 10_000

        ## user id
        self.user_embedding = tf.keras.Sequential([
                                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                                    vocabulary=unique_user_ids, mask_token=None),
                                                    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
                                                    ])
       
        # product category
        # self.catagory_embedding = tf.keras.Sequential([
        #                           tf.keras.layers.experimental.preprocessing.StringLookup(
        #                             vocabulary=unique_category, mask_token=None),
        #                           tf.keras.layers.Embedding(len(unique_category) + 1, self.embedding_dimension)
        #                         ])
        self.mood_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_mood, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_mood) + 1, self.embedding_dimension)
                                ])

        
        # self.catagory_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        #                                                                                 max_tokens=max_tokens)
        # self.catagory_text_embedding = tf.keras.Sequential([
        #                           self.catagory_vectorizer,
        #                           tf.keras.layers.Embedding(max_tokens, self.embedding_dimension, mask_zero=True),
        #                           tf.keras.layers.GlobalAveragePooling1D(),
        #                         ])

        # self.catagory_vectorizer.adapt(category)
              
        self.landscape_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_landscape, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_landscape) + 1, self.embedding_dimension)
                                ])
        self.DrivingStyle_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_DrivingStyle, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_DrivingStyle) + 1, self.embedding_dimension)
                                ])
        self.naturalphenomena_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_naturalphenomena, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_naturalphenomena) + 1, self.embedding_dimension)
                                ])
        self.RoadType_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_RoadType, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_RoadType) + 1, self.embedding_dimension)
                                ])
        self.sleepiness_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_sleepiness, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_sleepiness) + 1, self.embedding_dimension)
                                ])
        self.trafficConditions_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_trafficConditions, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_trafficConditions) + 1, self.embedding_dimension)
                                ])
        self.weather_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_weather, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_weather) + 1, self.embedding_dimension)
                                ])


        

    def call(self, inputs):
        # Take the input dictionary, pass it through each input layer,
        # and concatenate the result.
        return tf.concat([
            self.user_embedding(inputs["userID"]),
            self.DrivingStyle_embedding(inputs["DrivingStyle"]),
            self.landscape_embedding(inputs["landscape"]),
            self.naturalphenomena_embedding(inputs["naturalphenomena"]),
            self.RoadType_embedding(inputs["RoadType"]),
            self.sleepiness_embedding(inputs["sleepiness"]),
            self.trafficConditions_embedding(inputs["trafficConditions"]),
            self.weather_embedding(inputs["weather"]),
            self.mood_embedding(inputs["mood"])
        ], axis=1)

In [ ]:
class QueryModel(tf.keras.Model):
    """Model for encoding user queries."""

    def __init__(self, layer_sizes, projection_dim=None):
        """Model for encoding user queries
        """
        super().__init__()

        # We first use the user model for generating embeddings.
        self.embedding_model = UserModel()
            

        # Then construct the layers.
        self.dense_layers = tf.keras.Sequential()
     
      
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

        # No activation for the last layer.
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [ ]:
class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        self.embedding_dimension = 32

        #max_tokens = 10_000

        self.item_embedding = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
              vocabulary=unique_items,mask_token=None),
          tf.keras.layers.Embedding(len(unique_items) + 1, self.embedding_dimension)
        ])

        # self.item_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        #     max_tokens=max_tokens)

        # self.item_vectorizer.adapt(items)
      
    def call(self, it):
        return tf.concat([
            self.item_embedding(it)
         
        ], axis=1)

In [ ]:
class CandidateModel(tf.keras.Model):
    """Model for encoding items."""

    def __init__(self, layer_sizes, projection_dim=None):
        """Model for encoding items.

        
        """
        super().__init__()

        self.embedding_model = ItemModel()

         # Then construct the layers.
        self.dense_layers =  tf.keras.Sequential()
     

        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

        # No activation for the last layer.
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [ ]:
class RetrivalModel(tfrs.models.Model):

    def __init__(self, layer_sizes,projection_dim=None ):
        super().__init__()
        
        self.query_model : tf.keras.Model = QueryModel(layer_sizes)
        self.candidate_model : tf.keras.Model = CandidateModel(layer_sizes)
        
        ## rating and retrieval task.
        
        # self.rating_task = tfrs.tasks.Ranking(
        #     loss=tf.keras.losses.MeanSquaredError(),
        #     metrics=[tf.keras.metrics.RootMeanSquaredError()],
        # )
                 
        self.retrieval_task : tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=itemlists.batch(128).map(self.candidate_model)
            )
        )

    

    def compute_loss(self, features, training=False):
        
        # We only pass the user id and timestamp features into the query model. This
        # is to ensure that the training inputs would have the same keys as the
        # query inputs. Otherwise the discrepancy in input structure would cause an
        # error when loading the query model after saving it.
        ratings = features.pop("rating")
        
        query_embeddings = self.query_model({
            "userID": features["userID"],
            "DrivingStyle": features["DrivingStyle"],
            "landscape": features["landscape"],
            "naturalphenomena": features["naturalphenomena"],
            "RoadType": features["RoadType"],
            "sleepiness": features["sleepiness"],
            "trafficConditions": features["trafficConditions"],
             "weather": features["weather"],
            "mood": features["mood"]
        })
    
        item_embeddings = self.candidate_model(features["itemID"])       
        retrieval_loss = self.retrieval_task(query_embeddings, item_embeddings)
    
    
        return self.retrieval_task(query_embeddings, item_embeddings)

In [ ]:
cached_train = train.shuffle(train_size).batch(128).cache()
cached_test = test.batch(32).cache()

model = RetrivalModel([128,32], 
                      projection_dim=None)



In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01))

modelhist_dcn=model.fit(cached_train, 
         epochs=1)

metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Retrieval top-50 accuracy: {metrics['factorized_top_k/top_50_categorical_accuracy']:.3f}.")
print(f"Retrieval top-10 accuracy: {metrics['factorized_top_k/top_10_categorical_accuracy']:.3f}.")
print(f"Retrieval top-5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy']:.3f}.")
print(f"Retrieval top-1 accuracy: {metrics['factorized_top_k/top_1_categorical_accuracy']:.3f}.")

26/26 [==============================] - 3s 92ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0062 - factorized_top_k/top_5_categorical_accuracy: 0.0436 - factorized_top_k/top_10_categorical_accuracy: 0.0860 - factorized_top_k/top_50_categorical_accuracy: 0.4539 - factorized_top_k/top_100_categorical_accuracy: 0.8017 - loss: 101.9048 - regularization_loss: 0.0000e+00 - total_loss: 101.9048
Retrieval top-100 accuracy: 0.802.
Retrieval top-50 accuracy: 0.454.
Retrieval top-10 accuracy: 0.086.
Retrieval top-5 accuracy: 0.044.
Retrieval top-1 accuracy: 0.006.


In [ ]:

index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
index.index_from_dataset(
  tf.data.Dataset.zip((itemlists.batch(100), itemlists.batch(100).map(model.candidate_model)))
)
 
 
_, titles = index({
    "userID": np.array(['25']),
     "DrivingStyle": np.array(['NA']),
            "landscape":np.array(['NA']),
            "naturalphenomena": np.array(['NA']),
            "RoadType": np.array(['NA']),
            "sleepiness": np.array(['NA']),
            "trafficConditions": np.array(['NA']),
             "weather": np.array(['rainy']),
            "mood": np.array(['NA']
    )},
    k=10
)
titles[0]

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'288', b'678', b'286', b'277', b'746', b'761', b'759', b'706',
       b'729', b'691'], dtype=object)>

# Deep cross Network

## User model

In [ ]:
class UserModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        self.embedding_dimension = 32
        max_tokens = 10_000

        ## user id
        self.user_embedding = tf.keras.Sequential([
                                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                                    vocabulary=unique_user_ids, mask_token=None),
                                                    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
                                                    ])
       
        # product category
        # self.catagory_embedding = tf.keras.Sequential([
        #                           tf.keras.layers.experimental.preprocessing.StringLookup(
        #                             vocabulary=unique_category, mask_token=None),
        #                           tf.keras.layers.Embedding(len(unique_category) + 1, self.embedding_dimension)
        #                         ])
        self.mood_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_mood, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_mood) + 1, self.embedding_dimension)
                                ])

        
        # self.catagory_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        #                                                                                 max_tokens=max_tokens)
        # self.catagory_text_embedding = tf.keras.Sequential([
        #                           self.catagory_vectorizer,
        #                           tf.keras.layers.Embedding(max_tokens, self.embedding_dimension, mask_zero=True),
        #                           tf.keras.layers.GlobalAveragePooling1D(),
        #                         ])

        # self.catagory_vectorizer.adapt(category)
              
        self.landscape_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_landscape, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_landscape) + 1, self.embedding_dimension)
                                ])
        self.DrivingStyle_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_DrivingStyle, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_DrivingStyle) + 1, self.embedding_dimension)
                                ])
        self.naturalphenomena_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_naturalphenomena, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_naturalphenomena) + 1, self.embedding_dimension)
                                ])
        self.RoadType_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_RoadType, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_RoadType) + 1, self.embedding_dimension)
                                ])
        self.sleepiness_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_sleepiness, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_sleepiness) + 1, self.embedding_dimension)
                                ])
        self.trafficConditions_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_trafficConditions, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_trafficConditions) + 1, self.embedding_dimension)
                                ])
        self.weather_embedding = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                    vocabulary=unique_weather, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_weather) + 1, self.embedding_dimension)
                                ])


        

    def call(self, inputs):
        # Take the input dictionary, pass it through each input layer,
        # and concatenate the result.
        return tf.concat([
            self.user_embedding(inputs["userID"]),
            self.DrivingStyle_embedding(inputs["DrivingStyle"]),
            self.landscape_embedding(inputs["landscape"]),
            self.naturalphenomena_embedding(inputs["naturalphenomena"]),
            self.RoadType_embedding(inputs["RoadType"]),
            self.sleepiness_embedding(inputs["sleepiness"]),
            self.trafficConditions_embedding(inputs["trafficConditions"]),
            self.weather_embedding(inputs["weather"]),
            self.mood_embedding(inputs["mood"])
        ], axis=1)

In [ ]:
class QueryModel(tf.keras.Model):
    """Model for encoding user queries."""

    def __init__(self, layer_sizes, projection_dim=None):
        """Model for encoding user queries
        """
        super().__init__()

        # We first use the user model for generating embeddings.
        self.embedding_model = UserModel()
            

        # Then construct the layers.
        self.dense_layers = tf.keras.Sequential(tfrs.layers.dcn.Cross(projection_dim=projection_dim,
                                        kernel_initializer="glorot_uniform"))

        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

        # No activation for the last layer.
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [ ]:
class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        self.embedding_dimension = 32

        max_tokens = 10_000

        self.item_embedding = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
              vocabulary=unique_items,mask_token=None),
          tf.keras.layers.Embedding(len(unique_items) + 1, self.embedding_dimension)
        ])

        # self.item_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        #     max_tokens=max_tokens)

        # self.item_text_embedding = tf.keras.Sequential([
        #   self.item_vectorizer,
        #   tf.keras.layers.Embedding(max_tokens, self.embedding_dimension, mask_zero=True),
        #   tf.keras.layers.GlobalAveragePooling1D(),
        # ])

        # self.item_vectorizer.adapt(items)
      
    def call(self, items):
        return tf.concat([
            self.item_embedding(items),
            
        ], axis=1)

In [ ]:
class CandidateModel(tf.keras.Model):
    """Model for encoding items."""

    def __init__(self, layer_sizes, projection_dim=None):
        """Model for encoding items.

        
        """
        super().__init__()

        self.embedding_model = ItemModel()

         # Then construct the layers.
        self.dense_layers = tf.keras.Sequential(tfrs.layers.dcn.Cross(projection_dim=projection_dim,
                                                kernel_initializer="glorot_uniform"))

        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

        # No activation for the last layer.
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [ ]:
class CrossDNNModel(tfrs.models.Model):

    def __init__(self, layer_sizes,projection_dim=None ):
        super().__init__()
        
        self.query_model : tf.keras.Model = QueryModel(layer_sizes)
        self.candidate_model : tf.keras.Model = CandidateModel(layer_sizes)
        
        ## rating and retrieval task.
        
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
                 
        self.retrieval_task : tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=itemlists.batch(128).map(self.candidate_model)
            )
        )

      

    def compute_loss(self, features, training=False):
        
        # We only pass the user id and timestamp features into the query model. This
        # is to ensure that the training inputs would have the same keys as the
        # query inputs. Otherwise the discrepancy in input structure would cause an
        # error when loading the query model after saving it.
        ratings = features.pop("rating")
        
        query_embeddings = self.query_model({
            "userID": features["userID"],
            "DrivingStyle": features["DrivingStyle"],
            "landscape": features["landscape"],
            "naturalphenomena": features["naturalphenomena"],
            "RoadType": features["RoadType"],
            "sleepiness": features["sleepiness"],
            "trafficConditions": features["trafficConditions"],
             "weather": features["weather"],
            "mood": features["mood"]
        })
    
        item_embeddings = self.candidate_model(features["itemID"])       
        retrieval_loss = self.retrieval_task(query_embeddings, item_embeddings)
    
    
        return self.retrieval_task(query_embeddings, item_embeddings)

In [ ]:
cached_train = train.shuffle(train_size).batch(32).cache()
cached_test = test.batch(32).cache()

model = CrossDNNModel([128,32], 
                      projection_dim=None)



In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01))

modelhist_dcn=model.fit(cached_train, 
         epochs=3)

metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Retrieval top-50 accuracy: {metrics['factorized_top_k/top_50_categorical_accuracy']:.3f}.")
print(f"Retrieval top-10 accuracy: {metrics['factorized_top_k/top_10_categorical_accuracy']:.3f}.")
print(f"Retrieval top-5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy']:.3f}.")
print(f"Retrieval top-1 accuracy: {metrics['factorized_top_k/top_1_categorical_accuracy']:.3f}.")

Epoch 1/3
101/101 [==============================] - 13s 106ms/step - root_mean_squared_error: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0097 - factorized_top_k/top_5_categorical_accuracy: 0.0424 - factorized_top_k/top_10_categorical_accuracy: 0.0807 - factorized_top_k/top_50_categorical_accuracy: 0.3864 - factorized_top_k/top_100_categorical_accuracy: 0.7205 - loss: 109.2698 - regularization_loss: 0.0000e+00 - total_loss: 109.2698
Epoch 2/3
101/101 [==============================] - 11s 105ms/step - root_mean_squared_error: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0103 - factorized_top_k/top_5_categorical_accuracy: 0.0564 - factorized_top_k/top_10_categorical_accuracy: 0.1165 - factorized_top_k/top_50_categorical_accuracy: 0.4730 - factorized_top_k/top_100_categorical_accuracy: 0.7928 - loss: 107.0468 - regularization_loss: 0.0000e+00 - total_loss: 107.0468
Epoch 3/3
26/26 [==============================] - 3s 91ms/step - root_mean_squared_error: 0

In [ ]:

index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
index.index_from_dataset(
  tf.data.Dataset.zip((itemlists.batch(100), itemlists.batch(100).map(model.candidate_model)))
)
 
 
_, titles = index({
    "userID": np.array(['25']),
     "DrivingStyle": np.array(['NA']),
            "landscape":np.array(['NA']),
            "naturalphenomena": np.array(['NA']),
            "RoadType": np.array(['NA']),
            "sleepiness": np.array(['NA']),
            "trafficConditions": np.array(['NA']),
             "weather": np.array(['rainy']),
            "mood": np.array(['NA']
    )},
    k=10
)
titles[0]

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'249', b'263', b'279', b'250', b'719', b'716', b'262', b'740',
       b'762', b'286'], dtype=object)>